In [1]:
%load_ext autoreload
from path import Path2D
from interpolator import LinearInterpolator, NodeCollection, SplineInterpolator

import numpy as np
import matplotlib.pyplot as plt


In [ ]:
%autoreload 2
N = 100
x = np.linspace(0, 100, N)
y = np.ones(N)*30
t = np.linspace(0, 100, len(x))
tmax = t[-1]

nodes = NodeCollection(np.array([x, y]).T, t)
path = Path2D(nodes, SplineInterpolator)

nodes2 = NodeCollection(np.array([x, -y]).T, t)
path2 = Path2D(nodes2, LinearInterpolator)


In [ ]:
from metrics import RadarGenerator

nprobes = 200
probex = np.linspace(0, 0, nprobes)
probey = np.linspace(0, 0, nprobes)
probetimes = np.linspace(0.1, tmax, nprobes)
probes = np.array([probetimes, probex, probey]).T

radarGen = RadarGenerator([path, path2], 0.1, 1)
measurements = radarGen.make_measurement_series(probes)
# msdata = np.array([[ms.x, ms.y] for ms in measurements])


In [ ]:
from filters.jpda import JPDACF
import copy

%matplotlib qt

dt = 0.05
jpdaf = JPDACF(np.array([[0, 1, 30, 0], [0, 1, -30, 0]]), dt, 1)

T = tmax
steps = int(T / dt)

x1_k = []
x2_k = []
tspace = np.array([])
# msrmts = copy.deepcopy(measurements)
t = 0
for i in range(steps):
    t += dt

    jpdaf.predict()
    x1_k.append(jpdaf.positions[0])
    x2_k.append(jpdaf.positions[1])

    tspace = np.append(tspace, t)


# msdata = np.array([[ms.x, ms.y, ms.t] for ms in measurements])
x1 = np.array(x1_k)[:, 0]
y1 = np.array(x1_k)[:, 1]
x2 = np.array(x2_k)[:, 0]
y2 = np.array(x2_k)[:, 1]

plt.figure()
plt.scatter(x1, y1, color='red')
plt.scatter(x2, y2, color='green')
# plt.scatter(msdata[:, 0], msdata[:, 1], color='blue', s=5)
# data = np.array([path.pos(t) for t in np.linspace(0, T, 150)])
# plt.plot(data[:, 0], data[:, 1], color='green')


In [ ]:
from filters.kalman import KalmanFilter
import copy

%matplotlib qt

kf = KalmanFilter(np.array([0, 1, 0, 0, 0, 0]), 0.1)

dt = 0.05
T = tmax
steps = int(T / dt)

P_k = []
x_k = []
tspace = np.array([])
msrmts = copy.deepcopy(measurements)
t = 0
for i in range(steps):
    t += dt

    kf.predict(dt)
    P_k.append(kf.cov)
    x_k.append(kf.prediction)

    tspace = np.append(tspace, t)

    if (i+1)*dt >= msrmts[0].t:
        kf.update(np.array([msrmts[0].x, msrmts[0].y]), 1, 1)
        P_k.append(kf.cov)
        x_k.append(kf.prediction)
        tspace = np.append(tspace, msrmts[0].t)
        t = msrmts[0].t
        msrmts.pop(0)
        if len(msrmts) == 0:
            break

msdata = np.array([[ms.x, ms.y, ms.t] for ms in measurements])
x = np.array(x_k)[:, 0]
y = np.array(x_k)[:, 3]

plt.figure()
plt.subplot(2, 2, (1, 2))
plt.plot(x, y, color='red')
plt.scatter(x, y, color='red')
plt.scatter(msdata[:, 0], msdata[:, 1], color='blue', s=8)
data = np.array([path.pos(t) for t in np.linspace(0, T, 150)])
plt.plot(data[:, 0], data[:, 1], color='green')

Px = np.array(P_k)[:, 0, 0]
Py = np.array(P_k)[:, 3, 3]
plt.subplot(2, 2, (3))
plt.grid(True)
plt.plot(tspace, x, color='red')
plt.scatter(tspace, x, color='red', s=5)
plt.scatter(msdata[:, 2], msdata[:, 0], color='blue', s=8)
data = np.array([path.pos(t) for t in np.linspace(0, T, 150)])
plt.plot(np.linspace(0, T/dt, 150)*dt, data[:, 0], color='purple')


plt.show()
plt.subplot(2, 2, (4))
plt.plot(tspace, y, color='red')
plt.scatter(msdata[:, 2], msdata[:, 1], color='blue', s=8)
plt.plot(np.linspace(0, T/dt, 150)*dt, data[:, 1], color='purple')


In [3]:

import timeit
import numpy as np
from filters.jpda import Cluster, Track, Measurement, build_clusters, track_betas

# %autoreload 2

P = np.eye(4)
R = np.diag([1, 1])
H = np.array([[1, 0, 0, 0], [0, 0, 1, 0]])

x1 = np.array([-20.8, 0, 0, 0])
x2 = np.array([-20, 0, 0, 0])
x3 = np.array([-22, 0, 0.5, 0])

m1 = Measurement(1, np.array([21, 0]))
m2 = Measurement(2, np.array([-21, -2]))
m3 = Measurement(3, np.array([18, 0]))
m4 = Measurement(4, np.array([0, 0]))
m5 = Measurement(5, np.array([-19, 0]))
m6 = Measurement(6, np.array([-20, 0.5]))

t1 = Track(x1, P, H, R)
t2 = Track(x2, P, H, R)
t3 = Track(x3, P, H, R)


def fn():
    t1.measurement_selection([m1, m2, m3, m4, m5, m6], 4)
    t2.measurement_selection([m1, m2, m3, m4, m5, m6], 4)
    t3.measurement_selection([m1, m2, m3, m4, m5, m6], 4)

    clusters = build_clusters([t1, t2, t3])

    for clus in clusters:
        track_betas(clus.tracks)


N = 10000
elapsed_time = timeit.timeit(stmt=fn, setup=asdf, number=N)
print(f'elapsed time: {elapsed_time/N * 1000}')


NameError: name 't1' is not defined

track 0 betas: {0: 0.005710344783865143, 2: 2.07983295996227e-06, 5: 9.425960224576088e-06, 6: 0.0007009858526259818}
track 1 betas: {0: 0.0036611402426779955, 2: 1.954419016094061e-07, 5: 0.0005819054481788564, 6: 0.002179595296917202}
track 2 betas: {0: 0.006421435511767435, 2: 3.80790802974516e-09, 6: 1.3971100001991542e-06}

In [ ]:
# iu = [1, 2, 3, 4]
# O = [[0, 1, 3, 4], [0, 2, 3], [0, 1, 2, 4], [0, 1, 2, 3, 4]]
O = [{0,1,2,3,4}]
tau = 0
i = 2

Oc = O.copy()
Oc.pop(tau)
used = [] if i == 0 else [i]
if 0 in used:
    used.remove(0)
print(Oc)
print(used)


def enumerate_events(v, M, u, d, E):
    print(d, len(M))
    if (d) is (len(M)):
        E.append(v)
        return

    for i in M[d]:
        if i not in u:
            vnew = v.copy()
            vnew.append(i)
            unew = u.copy()
            if i != 0:
                unew.append(i)
            enumerate_events(vnew, M, unew, d+1, E)


In [ ]:
events = []
enumerate_events([], Oc, used, 0, events)
for e in events:
    e.insert(tau, i)
print(events)
print(len(events))